#  RAG 엔진 테스트 (Pinecone + Priority Rerank)

**목표**: 업로드된 Pinecone에서 검색 → Priority Rerank → GPT-4o 답변 생성

**이전 단계**: `embed_to_pinecone.ipynb` 완료 (CSV → Pinecone 업로드)

##  1단계: 라이브러리 임포트 및 API 키 로드

In [4]:
import os
from pathlib import Path
from dotenv import load_dotenv

from langchain_upstage import UpstageEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from pinecone import Pinecone

print("✅ 라이브러리 임포트 완료")

✅ 라이브러리 임포트 완료


In [5]:
# API 키 로드
BASE_DIR = Path.cwd().parents[1]  # 실프로젝트
env_path = BASE_DIR / ".env"
load_dotenv(env_path)

PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
UPSTAGE_API_KEY = os.getenv("UPSTAGE_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
INDEX_NAME = "real-estate-law-index"

# API 키 확인
print("🔑 API 키 확인:")
print(f"  PINECONE: {'✅' if PINECONE_API_KEY else '❌'}")
print(f"  UPSTAGE: {'✅' if UPSTAGE_API_KEY else '❌'}")
print(f"  OPENAI: {'✅' if OPENAI_API_KEY else '❌'}")
print(f"\n📌 인덱스: {INDEX_NAME}")

🔑 API 키 확인:
  PINECONE: ✅
  UPSTAGE: ✅
  OPENAI: ✅

📌 인덱스: real-estate-law-index


## 2단계: Upstage 임베딩 & Pinecone 연결

In [6]:
print("📦 Upstage 임베딩 로드 중...")
embeddings = UpstageEmbeddings(
    model="solar-embedding-1-large-passage",
    api_key=UPSTAGE_API_KEY
)
print("✅ 임베딩 준비 완료")

📦 Upstage 임베딩 로드 중...
✅ 임베딩 준비 완료


In [7]:
print(f" Pinecone 연결 중... (인덱스: {INDEX_NAME})")
pc = Pinecone(api_key=PINECONE_API_KEY)

vectorstore = PineconeVectorStore(
    index_name=INDEX_NAME,
    embedding=embeddings
)
print("✅ Pinecone 연결 완료")

 Pinecone 연결 중... (인덱스: real-estate-law-index)
✅ Pinecone 연결 완료


##  3단계: OpenAI LLM 초기화

In [8]:
print("🤖 OpenAI LLM 초기화 중...")
llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0.2,
    openai_api_key=OPENAI_API_KEY
)
print("✅ LLM 준비 완료")

🤖 OpenAI LLM 초기화 중...
✅ LLM 준비 완료


##  4단계: 프롬프트 템플릿 생성

In [9]:
template = """당신은 임차인의 권익을 보호하는 전문 법률 상담 AI입니다.

### 중요 규칙:
1. **반드시 근거 법령을 명시**하세요 (예: 주택임대차보호법 제3조)
2. 임차인에게 **불리한 조항은 명확히 경고**하세요
3. **구체적인 해결 방안**을 제시하세요
4. 법률 용어는 쉽게 풀어서 설명하세요

### 검색된 법령 및 사례:
{context}

### 사용자 질문:
{question}

### AI 답변:
"""

prompt_template = PromptTemplate(
    template=template,
    input_variables=["context", "question"]
)

print("✅ 프롬프트 템플릿 생성 완료")

✅ 프롬프트 템플릿 생성 완료


## 5단계: Priority Rerank 함수

In [10]:
def rerank_by_priority(documents):
    """
    우선순위 기반 rerank
    priority 낮을수록 가중치 높게
    
    순서:
    1. 특별법 (priority=1)
    2. 시행령 (priority=2)
    3. 시행규칙 (priority=3)
    4. 민법 (priority=4)
    5. 대법원 판례 (priority=5)|
    6. 하급심 판례 (priority=6)
    7. 상담 사례 (priority=7)
    """
    # priority로 정렬 (낮을수록 우선)
    reranked = sorted(
        documents,
        key=lambda doc: doc.metadata.get('priority', 99)
    )
    return reranked

print("✅ Rerank 함수 정의 완료")

✅ Rerank 함수 정의 완료


##  6단계: RAG 질의 함수

In [24]:
import textwrap
def query_rag(question, k=10):
    """
    질의에 대한 답변 생성
    
    Args:
        question: 사용자 질문
        k: 검색할 문서 개수 (많이 가져온 후 rerank)
    
    Returns:
        str: AI 답변
    """
    try:
        # 1. 유사도 검색 (많이 가져오기)
        print(f"🔍 유사도 검색 중... (k={k})")
        raw_docs = vectorstore.similarity_search(question, k=k)
        print(f"   검색 결과: {len(raw_docs)}개")
        
        # 2. 우선순위 rerank
        print(f"🔄 Priority Rerank 중...")
        reranked_docs = rerank_by_priority(raw_docs)
        
        # 3. 상위 5개만 사용
        top_docs = reranked_docs[:5]
        print(f"   상위 5개 선택 완료")
        
        # 4. Context 생성
        context_parts = []
        for i, doc in enumerate(top_docs, 1):
            meta = doc.metadata
            priority = meta.get('priority', '?')
            law_name = meta.get('law_name', 'Unknown')
            article = meta.get('article', '')
            
            content = textwrap.fill(doc.page_content, width=30)  # 밖으로!
    
            context_parts.append(
                f"[문서 {i}] (우선순위: {priority})\n"
                f"출처: {law_name} {article}\n"
                f"내용: {content}\n"
            )
        
        context = "\n".join(context_parts)
        
        # 5. LLM 답변 생성
        print(f"🤖 GPT-4o 답변 생성 중...")
        prompt = prompt_template.format(
            context=context,
            question=question
        )
        
        answer = llm.invoke(prompt).content
        
        # 줄 단위로 wrapping
        lines = answer.split('\n')
        wrapped_lines = []
        for line in lines:
            if line.strip():
                wrapped_lines.append(textwrap.fill(line, width=80))
            else:
                wrapped_lines.append('')
        answer = '\n'.join(wrapped_lines)
        # 6. 참고 법령 추가
        answer += "\n\n---\n**📚 참고 법령 및 사례:**\n"
        for i, doc in enumerate(top_docs, 1):
            meta = doc.metadata
            answer += (
                f"{i}. {meta.get('law_name', '?')} "
                f"{meta.get('article', '')} "
                f"(우선순위: {meta.get('priority', '?')})\n"
            )
        
        answer += "\n⚠️ 본 답변은 AI 분석 결과로, 최종 결정 전 변호사와 상담하시기 바랍니다."
        
        return answer
        
    except Exception as e:
        return f"❌ 오류가 발생했습니다: {str(e)}"

print("✅ RAG 질의 함수 정의 완료")

✅ RAG 질의 함수 정의 완료


##  7단계: 테스트 질의

In [25]:
# 테스트 질문 1
question1 = "보증금 반환을 안 해주면 어떻게 하나요?"

print("=" * 70)
print(f"Q: {question1}")
print("-" * 70)

answer1 = query_rag(question1)
print("\n" + answer1)

Q: 보증금 반환을 안 해주면 어떻게 하나요?
----------------------------------------------------------------------
🔍 유사도 검색 중... (k=10)
   검색 결과: 10개
🔄 Priority Rerank 중...
   상위 5개 선택 완료
🤖 GPT-4o 답변 생성 중...

보증금 반환 문제는 임차인에게 매우 중요한 사안입니다. 보증금을 돌려받지 못한 경우, 다음과 같은 절차를 통해 해결할 수 있습니다.

1. **임차권등기명령 신청**: 임대차가 종료된 후에도 보증금을 돌려받지 못한 경우, 임차인은 임차주택의 소재지를 관할하는 법원에
임차권등기명령을 신청할 수 있습니다. 이를 통해 임차권등기를 완료하면 대항력과 우선변제권을 취득할 수 있습니다. (주택임대차보호법 제3조의3)

2. **보증금반환청구소송**: 임차인은 보증금 반환을 위해 법원에 소송을 제기할 수 있습니다. 소송을 통해 확정판결을 받으면 경매를 신청하여
보증금을 회수할 수 있습니다. (주택임대차보호법 제3조의2)

3. **경매 신청**: 보증금반환청구소송의 확정판결을 받은 후, 임차인은 경매를 통해 보증금을 회수할 수 있습니다. 이 경우, 임차인은
후순위권리자보다 우선하여 보증금을 변제받을 권리가 있습니다. (주택임대차보호법 제3조의2)

4. **임차권등기명령 비용 청구**: 임차권등기명령과 관련된 비용은 임대인에게 청구할 수 있습니다. (주택임대차보호법 제3조의3 제8항)

5. **전세대출 연장**: 만약 전세대출이 있는 경우, 대출보증기관의 연장 방안을 통해 대출을 연장할 수 있습니다. 임차권등기 신청 접수증을
통해 6개월 연장이 가능하며, 임차권등기 경료 이후 추가 연장이 가능합니다. (상담사례집 사례271)

이러한 절차를 통해 보증금 반환 문제를 해결할 수 있으며, 필요시 법률 전문가의 도움을 받는 것도 고려해보시기 바랍니다.

---
**📚 참고 법령 및 사례:**
1. 주택임대차보호법 제3조의3 (우선순위: 1.

In [26]:
# 테스트 질문 2
question2 = "천장에서 물이 새는데 집주인이 수리를 안 해줘요"

print("=" * 70)
print(f"Q: {question2}")
print("-" * 70)

answer2 = query_rag(question2)
print("\n" + answer2)

Q: 천장에서 물이 새는데 집주인이 수리를 안 해줘요
----------------------------------------------------------------------
🔍 유사도 검색 중... (k=10)
   검색 결과: 10개
🔄 Priority Rerank 중...
   상위 5개 선택 완료
🤖 GPT-4o 답변 생성 중...

천장에서 물이 새는 경우, 이는 임대차 계약에서 임대인이 해결해야 할 문제입니다. 임대인은 임대물의 보존에 필요한 수리를 해야 할 의무가
있습니다. 이에 대한 법적 근거는 민법 제634조와 제624조입니다.

1. **민법 제634조**에 따르면, 임차물에 문제가 발생했을 때 임차인은 이를 임대인에게 즉시 통지해야 합니다. 따라서, 이미 통지하셨다면
임대인이 수리를 진행해야 합니다.

2. **민법 제624조**에 따르면, 임대인은 임대물의 보존에 필요한 행위를 할 수 있으며, 임차인은 이를 거절할 수 없습니다. 이는 임대인이
수리를 해야 한다는 의미입니다.

### 해결 방안:
1. **임대인에게 통지**: 이미 통지하셨다면, 이를 증명할 수 있는 문자나 이메일 등의 기록을 보관하세요. 아직 통지하지 않으셨다면, 즉시
임대인에게 문제를 알리세요.

2. **임대인의 수리 거부 시**: 임대인이 수리를 거부하거나 지연할 경우, 임차인은 법적 조치를 고려할 수 있습니다. 이 경우,
주택임대차보호법에 따라 임대차 계약을 해지하거나 손해배상을 청구할 수 있습니다.

3. **임시 조치**: 수리가 지연되는 동안 피해를 최소화하기 위해 임시 방수 조치를 고려하세요. 이 경우, 비용에 대한 영수증을 보관하여
나중에 임대인에게 청구할 수 있습니다.

4. **법적 조치**: 임대인이 계속해서 수리를 거부한다면, 법률 전문가와 상담하여 임대차 계약 해지나 손해배상 청구 등의 법적 조치를
고려하세요.

임대인이 수리를 거부하는 것은 임차인의 권리를 침해하는 행위이므로, 적극적으로 대응하시기 바랍니다.

---
**📚 참고 법령 및 

In [27]:
# 테스트 질문 3
question3 = "계약서에 '모든 수선 비용은 임차인이 부담한다'고 써있어요"

print("=" * 70)
print(f"Q: {question3}")
print("-" * 70)

answer3 = query_rag(question3)
print("\n" + answer3)

Q: 계약서에 '모든 수선 비용은 임차인이 부담한다'고 써있어요
----------------------------------------------------------------------
🔍 유사도 검색 중... (k=10)
   검색 결과: 10개
🔄 Priority Rerank 중...
   상위 5개 선택 완료
🤖 GPT-4o 답변 생성 중...

계약서에 "모든 수선 비용은 임차인이 부담한다"는 조항이 포함되어 있다면, 이는 임차인에게 불리한 조항일 수 있습니다. 일반적으로 임대인은
임차인이 임차물을 사용할 수 있도록 필요한 상태를 유지할 의무가 있습니다. 이는 민법 제623조에 명시되어 있습니다. 따라서 임대인은 임차물의
기본적인 수리와 유지보수를 책임져야 합니다.

**경고:** 이러한 조항은 임차인에게 불리할 수 있으며, 임대인의 의무를 임차인에게 전가하는 것으로 해석될 수 있습니다.

**해결 방안:**
1. **임대인과 협의:** 먼저 임대인과 협의하여 수선 비용 부담에 대한 조항을 재검토하고, 임대인이 기본적인 수리 의무를 지도록 계약을
수정하는 것이 좋습니다.

2. **법적 자문:** 필요하다면 법률 전문가의 도움을 받아 계약서의 조항을 검토하고, 임차인에게 불리한 조항을 수정할 수 있는 방법을 찾는
것이 좋습니다.

3. **계약서 수정:** 임대인과의 협의가 원활하지 않다면, 계약서에 명시된 수선 비용 부담 조항을 명확히 하고, 임대인의 수리 의무를
명시하는 방향으로 계약서를 수정하는 것을 고려해 보세요.

이러한 조치를 통해 임차인의 권익을 보호할 수 있습니다.

---
**📚 참고 법령 및 사례:**
1. 주택임대차보호법 제6조의3 (우선순위: 1.0)
2. 주택임대차보호법 제7조 (우선순위: 1.0)
3. 민법 제634조 (우선순위: 4.0)
4. 민법 제623조 (우선순위: 4.0)
5. 민법 제630조 (우선순위: 4.0)

⚠️ 본 답변은 AI 분석 결과로, 최종 결정 전 변호사와 상담하시기 바랍니다.


##  8단계: 검색 결과 상세 분석

In [28]:
# 검색 결과 상세 보기
test_query = "대항력 취득 요건"

print(f"🔍 질의: {test_query}")
print("="*70)

# 유사도 검색
raw_docs = vectorstore.similarity_search(test_query, k=10)

# Rerank 전
print("\n📊 Rerank 전 (유사도 순):")
for i, doc in enumerate(raw_docs[:5], 1):
    meta = doc.metadata
    print(f"[{i}] Priority {meta.get('priority', '?')} | "
          f"{meta.get('law_name', '?')} {meta.get('article', '')}")

# Rerank 후
reranked = rerank_by_priority(raw_docs)
print("\n🔄 Rerank 후 (우선순위 순):")
for i, doc in enumerate(reranked[:5], 1):
    meta = doc.metadata
    print(f"[{i}] Priority {meta.get('priority', '?')} | "
          f"{meta.get('law_name', '?')} {meta.get('article', '')}")

🔍 질의: 대항력 취득 요건

📊 Rerank 전 (유사도 순):
[1] Priority 1.0 | 주택임대차보호법 제3조의3
[2] Priority 1.0 | 주택임대차보호법 제3조의2
[3] Priority 1.0 | 주택임대차보호법 제3조의4
[4] Priority 1.0 | 주택임대차보호법 제3조
[5] Priority 7.0 | 상담사례집 사례1

🔄 Rerank 후 (우선순위 순):
[1] Priority 1.0 | 주택임대차보호법 제3조의3
[2] Priority 1.0 | 주택임대차보호법 제3조의2
[3] Priority 1.0 | 주택임대차보호법 제3조의4
[4] Priority 1.0 | 주택임대차보호법 제3조
[5] Priority 1.0 | 주택임대차보호법 제3조의6


##  9단계: 대화형 질의 (선택사항)

In [29]:
# 자유롭게 질문 입력
user_question = input("질문을 입력하세요: ")

if user_question:
    print("\n" + "="*70)
    print(f"Q: {user_question}")
    print("-"*70)
    
    answer = query_rag(user_question)
    print("\n" + answer)


Q: 입주한지 32개월인데 갑자기 나가라고 합니다
----------------------------------------------------------------------
🔍 유사도 검색 중... (k=10)
   검색 결과: 10개
🔄 Priority Rerank 중...
   상위 5개 선택 완료
🤖 GPT-4o 답변 생성 중...

임차인의 권익을 보호하기 위해 몇 가지 중요한 사항을 안내드리겠습니다.

1. **계약 갱신 요구권**: 주택임대차보호법 제6조의3에 따르면, 임차인은 계약 갱신을 요구할 권리가 있습니다. 임대인은 정당한 사유 없이
이를 거절할 수 없습니다. 정당한 사유에는 임차인이 2기의 차임액을 연체한 경우, 임차인이 거짓이나 부정한 방법으로 임차한 경우 등이
포함됩니다. 만약 이러한 사유가 없다면 임대인은 갱신 요구를 거절할 수 없습니다.

2. **계약 해지 통지**: 주택임대차보호법 제6조의2에 따르면, 임차인은 언제든지 임대인에게 계약 해지를 통지할 수 있으며, 임대인이 그
통지를 받은 날부터 3개월이 지나면 그 효력이 발생합니다. 반대로, 임대인이 임차인에게 나가라고 요구할 경우, 임대차 계약이 갱신되지 않았거나
정당한 해지 사유가 있어야 합니다.

3. **보증금 반환**: 주택임대차보호법 제4조에 따르면, 임대차 기간이 끝난 경우에도 임차인이 보증금을 반환받을 때까지는 임대차 관계가
지속됩니다. 따라서 보증금이 반환되지 않은 상태에서 임대인이 나가라고 요구하는 것은 부당할 수 있습니다.

### 해결 방안:
- **임대인과 대화**: 먼저 임대인과 대화를 통해 왜 나가라고 하는지 이유를 명확히 하세요. 정당한 사유가 있는지 확인하는 것이 중요합니다.
- **법적 조언 구하기**: 만약 임대인이 정당한 사유 없이 나가라고 요구한다면, 법률 전문가의 조언을 구하는 것이 좋습니다. 필요시 법적
절차를 통해 권리를 보호받을 수 있습니다.
- **계약 갱신 요구**: 정당한 사유가 없다면 계약 갱신 요구권을 행사하여 임대차 관계

## ✅ 완료!

**다음 단계**:
1. Streamlit 챗봇 UI 구현
2. 성능 평가 (RAGAS)
3. 프롬프트 개선

**참고**:
- Priority Rerank가 제대로 작동하는지 8단계에서 확인!
- 특별법(1) → 시행령(2) → ... → 사례(7) 순으로 정렬됨